# Projeto 1 - Ciência dos Dados

Nome: Mia Valentin Machado

Nome: Matheus Castellucci

Nome: Pedro Henrique

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\spedr\Desktop\2º Semestre\Dados\Projeto 1\Classificador


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
PS5 = 'PS5 - Final.xlsx'

In [4]:
train = pd.read_excel(PS5)
train.head(5)

,Tweet,Classificação de Relevância
0,um ps5 eh pra sempre https://t.co/ow1aobrsmm,2
1,"vi uma galerinha falando: ""ain que maravilha w...",0
2,"@jisegirl os dois, você no sofa, o ps5 no rack...",0
3,vender meu pulmão pra comprar um ps5 https://t...,2
4,@midiasonysta @playstation_br endosso o coro.....,1


In [5]:
test = pd.read_excel(PS5, sheet_name = 'Teste')
test.head(5)

,Tweet,Classificação de Relevância
0,"@wagnerthomazoni diria que é bom até, mais tem...",2
1,@d_ultra_ @esportzy @vplportugalps eu já que s...,1
2,@guisouzadias branco com ps5 em casa adora def...,0
3,@oblivionhell jogando no meu ps5,2
4,"queria pegar um console mais barato, mas exclu...",3


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é o PS5, console mais recente da Sony, uma empresa predominante no mercado de eletrônicos. Diante disto, consideramos como relevantes tweets com teor critico a cerca do console e suas particuliaridades, classificando então as opniões sobre seus jogos e manisfestações quanto ao desejo de possuir um PS5, como tweets irrelevantes ou neutros dependendo do teor do tweet que os um possível comprador.

-**Muito irrelevante**: Consideramos assim, os tweets referentes a ofertas de PS5 em lojas, links para sites com informações do PS5S ou vídeos que falavam sobre o mesmo, além de anúncios de sorteios do console.

-**Irrelevante**: Foram tweets que tinham um foco maior em otras coisas além do PS5, mas que comentavam sobre o console, fosse para ser um comparativo ou um meio de difamar alguém. Além disso, tweets falando especificamente de um jogo de PS5 sem falar nada do console, tambem foram considerados assim.

-**Neutro**: Tweets em que pessoas falavam que queriam um PS5, só que comentários irrelevantes atrelados. Tambem foram considerados assim, tweets que focavam principalmente em um jogo exclusivo do PS5 e como ele foi importante para a decisão sobre o PS5

-**Relevante**: Foram considerados assim, tweets que exaltavam as qualidades do PS5, seja comentando dele apenas ou das qualidades técnicas dos jogos. Ou críticas relevantes sobre algumas de suas características, sendo uma das principais, o preço do console dentro do Brasil.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def limpa(tweet_sujo):#Recebe o tweet breuto e o retorna sem caracteres especiais e com espaçamentos corretos em relação aos emojis
    
    tweet_sem_carac = re.sub(re.compile('[!-.:?;\n|]'), '', tweet_sujo)
    #Retira os caracteres especiais e coloca um ' ' no lugar
    tweet_limpo = re.sub(re.compile(r'(http\S*\s)'), '', tweet_sem_carac)
    #Retira os links e coloca um ' ' no lugar
    
    texto_prov = ''
    for palavra in tweet_limpo:   #Verifica cada palavra do tweet para verificar se ela é um emoji
        if palavra in emoji.UNICODE_EMOJI:
            texto_prov += f' {palavra} ' #Caso texto_prov seja um emoji a função o separará do texto com espaços.
        else:
            texto_prov += f'{palavra}'   #Caso não o texto continuará sem alteração
    tweet_limpo = texto_prov
    
    tweet_limpo = re.sub(re.compile(r'\s+'), ' ', tweet_limpo) 
    #Se existirem mais de um espaço seguidos essa linha transformará em apenas um espaço

    return tweet_limpo

classes =  [('mt_ir', 0), ('ir', 1), ('nt', 2), ('rl', 3)]     
#Nomes das classificações e os números em que os tweets foram classificados
nomes = {'mt_ir': 'Muito Irrelevante', 'ir': 'Irrelevante', 'nt': 'Neutro', 'rl': 'Relevante'}  
#Nomes reais das Classificações

tweet_classe = {}
#Dicionário que vai separar os tweets por classificação
for classe, num_class in classes:
    lista_prov = train.loc[train['Classificação de Relevância'] == num_class, :]      
    #Escolhe os tweets de uma só classificação
    tweet_classe[classe] = ''
    #Lista que receberá os tweets
    for tweet in lista_prov['Tweet']:
        tweet_classe[classe] += ' ' + tweet + ' ' 
        #adicionar os tweets de uma classificação a um item do dicionário
    
    tweet_classe[classe] = limpa(tweet_classe[classe].lower())
    #Limpa os tweet

tweet_classe['mt_ir'][:1000]

' vi uma galerinha falando ain que maravilha wolverine e spiderman 2 exclusivos ps5 chupa microsoft e caixistasantes ain starfield e elder scrolls 6 não podem ser exclusivos temos que fazer uma petiçãoow sério aqui vão se foder @jisegirl os dois você no sofa o ps5 no rack e nós tirando x1 no mortal kombat 11 vídeo playstationfifa 22 official pro clubs trailer ps5 ps4link “eu ou um ps5” dá uma tristeza em saber que deathloop é jogo exclusivo do ps5quem quiser me doar o ps5 😳 @bruno_cves o ps5 vem de brinde @lord_psmg @insomniacgames microsoft não valorizou o estúdio quando pode na época pós sunset overdrive agora se fode vendo ele empurar o ps5 @la3b_ps5 vigor @junglovelyy vc tem um ps5 @vitorr117 @andre_d4rk2 @profechinho @pedroxbz quando teve o teardown do ps5 o xbox fez um tweet dizendo como virar seu xbox e postou a foto de um series x em pé e na outra deitadochamaram o ps5 de playstation vent na cara larga num tweet do among us só nao te mostro pq faz mt mt mttt tempo velho novo tr

In [7]:
t = 0
#Número palavras por classe
for classe, n in classes:
    t += len(pd.Series(tweet_classe[classe].split()).value_counts())
for classe, n in classes:
    print(f'palavras na classe {nomes[classe]}:',len(pd.Series(tweet_classe[classe].split()).value_counts()), f'({(len(pd.Series(tweet_classe[classe].split()).value_counts())/t)*100:.3g}%)')

palavras na classe Muito Irrelevante: 1192 (30.6%)
palavras na classe Irrelevante: 860 (22.1%)
palavras na classe Neutro: 801 (20.5%)
palavras na classe Relevante: 1047 (26.8%)


In [8]:
alpha = 1       #constante alpha para ser utilizada caso precisemos usar uma palavra que não consta na base de dados
V = 10**6       #constante V para representar a quantidade de palavras póssiveis de usarmos em tweets

valores = {}    #dicionário que vai conter um DataFrame para cada classe informando o peso de cada palavra na base de dados da classe
for classe in tweet_classe: #conversão das strings para o DataFrame dos pesos
    valores[classe] = (pd.Series(tweet_classe[classe].split()).value_counts() + alpha) / (len(tweet_classe[classe].split()) + alpha*V)

valores['mt_ir'].head()

ps5    0.000126
de     0.000075
e      0.000068
o      0.000066
um     0.000058
dtype: float64

In [9]:
def classifica(tweet):
    '''
    Função que usa a base de dados da variável valores para classificar tweets na forma de uma única string em positivo, negativo ou neutro.
    '''
    tweet_limpo = limpa(tweet)
    frase_list = tweet_limpo.split()                        #lista com cada palavra da frase
    resultados = []                                         #lista que vai compreender uma tupla (classe, peso do tweet na respectiva classe) para cada classe
    for classe, n in classes:                               #uma vez para cada classe
        value = 0                                           #variavel que copreende a combinação dos pesos de cada palavra do tweet na classe da vez
        for palavra in frase_list:                          #uma vez para cada palavra no tweet
            if palavra in valores[classe]:                  #se a palavra constar na base de dados...
                value += np.log(valores[classe][palavra])   #adiciona o peso dela na variavel value
                #print(classe, palavra, np.log(valores[classe][palavra]), '(tem)')
            else:
                value += np.log(alpha / (len(tweet_classe[classe].split()) + alpha*V)) #adiciona o peso de uma palavra genérica na variavel value
                #print(classe, palavra, np.log(alpha / (len(tweet_classe[classe].split()) + alpha*V)), '(NAO tem)')
        resultados += [(value, n)]                          #adiciona a tupla (classe, peso do tweet na respectiva classe) na lista de resultados
    classificacao = (-10000, 0)                                  #variavel que vai compreender a tupla com o maior peso
    for value, n in resultados:                             #uma vez par cada tupla na lista de resultados
        if classificacao[0] < value:                        #se o peso dessa tupla for maior do que o da variavel classificação...
            classificacao = (value, n)                      #a variavel classificação assumi o valor dessa tupla
    
    return classificacao[1]                                 #retorna apenas o número da classifição alcançada

In [10]:
print(classifica('felipe neto'))

1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
perf = {} #dicionário que vai compreender os fesultados da verificação de performance
for classe, n in classes:
    perf[classe] = {}
    for classe_B, n_B in classes:
        perf[classe][classe_B] = 0
    frame = test.loc[train['Classificação de Relevância'] == n, :] #seleciona os tweets da planilha de teste da respectiva classe
    for tweet in frame['Tweet']:
        perf[classe][classes[classifica(tweet)][0]] += 1
perf

{'mt_ir': {'mt_ir': 21, 'ir': 4, 'nt': 22, 'rl': 92},
 'ir': {'mt_ir': 21, 'ir': 6, 'nt': 14, 'rl': 78},
 'nt': {'mt_ir': 15, 'ir': 3, 'nt': 22, 'rl': 101},
 'rl': {'mt_ir': 10, 'ir': 2, 'nt': 16, 'rl': 73}}

In [12]:
perf_clean = {'total':{'verdadeiro':0, 'falso':0}} #variável perf pronta para expor os dados no print
for classe, n in classes:
    perf_clean[classe] = {'verdadeiro':0, 'falso':0}
    for classe_B, n_B in classes:
        if classe == classe_B:
            perf_clean[classe]['verdadeiro'] += perf[classe_B][classe]
            perf_clean['total']['verdadeiro'] += perf[classe_B][classe]
        else:
            perf_clean[classe]['falso'] += perf[classe_B][classe]
            perf_clean['total']['falso'] += perf[classe_B][classe]

perf_clean

{'total': {'verdadeiro': 122, 'falso': 378},
 'mt_ir': {'verdadeiro': 21, 'falso': 46},
 'ir': {'verdadeiro': 6, 'falso': 9},
 'nt': {'verdadeiro': 22, 'falso': 52},
 'rl': {'verdadeiro': 73, 'falso': 271}}

In [13]:
for classe, n in classes: #print dos resultados da verificação de performance
    print(f'{nomes[classe]}:\n Verdadeiros ' + nomes[classe] + f': {perf_clean[classe]["verdadeiro"]} ({(perf_clean[classe]["verdadeiro"] / sum(perf_clean["total"].values()))*100:.3g}%)\n Falsos ' + nomes[classe] + f': {perf_clean[classe]["falso"]} ({(perf_clean[classe]["falso"] / sum(perf_clean["total"].values()))*100:.3g}%)')
print(f'Total:\n Verdadeiros total: {perf_clean["total"]["verdadeiro"]} ({(perf_clean["total"]["verdadeiro"] / sum(perf_clean["total"].values()))*100:.3g}%)\n Falsos total: {perf_clean["total"]["falso"]} ({(perf_clean["total"]["falso"] / sum(perf_clean["total"].values()))*100:.3g}%)')

Muito Irrelevante:
 Verdadeiros Muito Irrelevante: 21 (4.2%)
 Falsos Muito Irrelevante: 46 (9.2%)
Irrelevante:
 Verdadeiros Irrelevante: 6 (1.2%)
 Falsos Irrelevante: 9 (1.8%)
Neutro:
 Verdadeiros Neutro: 22 (4.4%)
 Falsos Neutro: 52 (10.4%)
Relevante:
 Verdadeiros Relevante: 73 (14.6%)
 Falsos Relevante: 271 (54.2%)
Total:
 Verdadeiros total: 122 (24.4%)
 Falsos total: 378 (75.6%)


___
### Concluindo

Como podemos ver na parte anterior do notebook, nosso classificador interpretar corretamente apenas 24.4% dos tweets da planilha Teste. Uma precisão de apenas 24.4% não é muito boa. Apesar do classificador Naive Bayes poder chegar a uma eficiencia de até 92%. Acreditamos que tal descrepância entre as porcentagens vem dos fatos de diversos tweets serem dificeis de serem avaliados, muitos deles demostram algo muito superfulo, algumas vezes até fútil. O que acaba fazenddo com que eficiencia caia. Outro motivo relevante para esse diferença de valore se encontra no tamanho da base de dados que foi usado, sendo consideravelmente bem menor do que o ideal para algo desta magnitude.

Após colocar o classificador para rodar, o resultado quanto a classificação de tweets "Muito Relevante" foi que: foram encontrados 4.2% corretamente classificados, enquanto 9.2% foram classificados erroniamente.
Para a classificação dos tweets "Irrelevantes" encontramos 1.2% corretamente e 1.8% classificados.
Já para os tweets "Neutros", encontramos 4.4% corretos e 10.4% incorretos, sendo aqui um começo do problema do nosso classificador, aqui encontramos uma diferença muito maior que nos outros dois.
Quando passamos para a análise dos tweets "Relevantes" percebemos onda encontra-se o problema do nosso classificador, para essa classe foram encontrados apennas 14.6% classificados corretamente, por outyro lado, 54.2% foram classificados de maneira errada.

Com base nessa experiência com o classificador Naive-Bayes, é possível entender o por que dele não pode ser usado para criar uma nova base de dados, sendo este motivo o de que, esse classificador iria criar uma base de dados maior do que deveria, isso porque iria criar uma parte errada. Ou seja, ele não se mostra viavel por criar uma base de dados com erros de classificação.

Pensando nisso, um possível meio em que o classificador Naive-Bayes ainda poderia ser aplicado, seria numa analise de mercado para algum tipo de produto. Sendo que iria apenas analisar para quais clientes seriam mostradas propagandas dele. Ou seja, algo que não existe muito problema em se errar a classificação, nesse caso, so mostraria a propaganda para alguém sem interesse.

Tendo tudo isso em vista, principalmentes a nossa experiência com o classificador. Uma possível melhoria para o classificador, seria atribuir para palvras a informação que a própria palavra atribua um valor para o todo em que esteja, assim não classificando errado as partes que têm essa palavra.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

O grupo não fez esta parte do projeto.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**